# Non negative Matrix Factorization from scratch

This notebook, we will implement the **Non negative Matrix Factorization** From scratch. I will try to compare two implementation. The first one uses the vanilla **numpy**. For the second,we will accelerate the process by calling all the computations on `gpu`s using **pytorch**.

<img src="images/nmf_doc.png" alt="NMF on documents" style="width: 80%"/>

[NMF tutorial](../documents/NMF_tutorial_ICME-2014.pdf)



## Group news

In this notebook, we will apply the **PCA** and **NMF** to compute those factorization

In [50]:
import numpy as np
from sklearn import decomposition
from sklearn.datasets import fetch_20newsgroups
import scipy.linalg as linalg
import matplotlib.pyplot as plt

In [51]:
%matplotlib inline
np.set_printoptions(suppress=True)

## Fetching the news

In [52]:
#list of categories and things to remove
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
remove = ('headers', 'footers', 'quotes')

newsgroup_train = fetch_20newsgroups(subset='train', categories = categories, remove = remove)
newsgroup_test = fetch_20newsgroups(subset='test', categories= categories, remove =  remove)

In [53]:
#printing information about newgropus
print("   categories are",newsgroup_train.target_names)
print("   Size of the documents", newsgroup_train.filenames.shape)
print("   Single example",newsgroup_train.data[0])

   categories are ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
   Size of the documents (2034,)
   Single example Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych


## TF-IDF matrix 

We will compute the **TF-IDF** matrix for each document. This matrix will serve a the main matrix $\mathbf{V}$

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [55]:
vectorizer = TfidfVectorizer(stop_words='english')
vectors_tf_idf=vectorizer.fit_transform(newsgroup_train.data).todense()  # We use todense since the method return a scipy  sparse matrix

In [56]:
print("    Shape of the vectors is:",vectors_tf_idf.shape)

    Shape of the vectors is: (2034, 26576)


# NMF with numpy

We will use **SGD** to compute the decompostion. The method used by *sklearn* minimize the following function:

$$
E(W,H) = \Vert R - W*H\Vert_2 + \lambda \big(  \alpha ( \Vert H \Vert_1 + \Vert W\Vert_1 ) + (1-\alpha)( \Vert H \Vert_{fro} + \Vert W\Vert_{fro}) \big)
$$

For the objectif function we will only consider the **Frobenieus** Norm for simplification. This choice conform with the value  of $\alpha=0$. in the previous objectif function.

$$
E(W,H) = \Vert R - W*H\Vert_2 + \lambda \big(  \Vert H \Vert_{fro} + \Vert W\Vert_{fro} \big)
$$

In order to compute the matrices $W$ and $H$. We will start with random matrices and iteratively apply the **SGD** to minimize the objectif function.

In [57]:
#shapes
m,n = vectors_tf_idf.shape
d=5  # num topics

In [58]:
#Initialisations
W = np.random.randn(m,d)
H = np.random.randn(d,n)

In [59]:
#Now let's  define our loss
R = vectors_tf_idf   # simple name for the tf_idf
def loss(W,H,lam=1):
    """
    Compute the loss of the matrix W,H
    """
    return np.linalg.norm(R - W@H) + lam* ( np.linalg.norm(W,ord='fro')+ np.linalg.norm(H,ord='fro'))

print("    Initial loss is : {:.5f}".format(loss(W,H)))

    Initial loss is : 17114.88842


## Gradient according to each matrix

In [96]:
def penalty(W,tol=1e-15):
    """
    Compute the error according the frobenius norm
    """
    return np.where(W>tol, 0 , np.min(W-tol,0))
def gradients(W,H,lam=1):
    """
    Compute the gradient of the loss function 
    return dW, dH
    """
    
    #compute the difference
    err= W@H - R
    
    return err@H.T+ lam*penalty(W), W.T@err + lam*penalty(H)

dW, dH = gradients(W,H)
print("   ",dW.shape)
print("   ",dH.shape)

    (2034, 5)
    (5, 26576)


/home/anass/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """


## Gradient descent

Now that we computed the loss function and the gradients, we will iteratively apply the gradient descent

In [97]:
lr_rate = 0.1
def update(W,H,lr=lr_rate):
    """
    Function to performe one iteration of SGD
    """
    #compute the gradients
    dW, dH = gradients(W,H)
    
    W -= lr * dW
    H -= lr * dH
    
    return W,H
    
def report(W,H):
    """
    Function to plot the statistics about the matrix
    """
    loss = np.linalg.norm(R- W@H)
    print("    loss: {:.4f}, min values = {:.2f},{:.2f}, negative values = {:d}, {:d} ".format(loss, W.min(),H.min(), np.sum(W<0),np.sum(H<0)))

# Initialisation

In [98]:
W, H = np.abs(np.random.normal(scale=0.01, size=(m,d))), np.abs(np.random.normal(scale=0.01, size=(d,n)))

In [89]:
report(W,H)

    loss: 44.4267, min values = 0.00,0.00, negative values = 0, 0 


In [92]:
W, H = update(W, H)
report(W, H)

    loss: 44.5484, min values = -0.01,-0.02, negative values = 557, 1918 


In [100]:
for i in range(50):
    W, H = update(W,H)
    
    if i % 10== 0:
        report(W,H)


    loss: 43.7546, min values = -0.22,-0.24, negative values = 1744, 51527 
    loss: 43.7458, min values = -0.22,-0.22, negative values = 1697, 53328 
    loss: 43.7398, min values = -0.20,-0.23, negative values = 1579, 53645 
    loss: 43.7356, min values = -0.19,-0.26, negative values = 1588, 52952 
    loss: 43.7319, min values = -0.17,-0.29, negative values = 1514, 52448 


Let's check the values by showing the topics

In [103]:
num_top_words=8
vocab = np.array(vectorizer.get_feature_names())
def show_topics(a):
    """
    Function to return the num_top_words in a topic
    """
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [105]:
show_topics(H)

['space don people nasa think like know just',
 'thanks graphics files image file program windows format',
 'objective morality values just moral science graphics think',
 'space ico bobbe tek bronx beauchaine vice manhattan',
 'god jesus bible people believe say atheism does']

Pretty good result. But the implementation is slow. We will accelerate this implementation using `pytorch`

## Pytorch

[PyTorch](http://pytorch.org/) is a Python framework for tensors and dynamic neural networks with GPU acceleration.  Many of the core contributors work on Facebook's AI team.  In many ways, it is similar to Numpy, only with the increased parallelization of using a GPU.

From the [PyTorch documentation](http://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html):

<img src="images/what_is_pytorch.png" alt="pytorch" style="width: 80%"/>

**Further learning**: If you are curious to learn what *dynamic* neural networks are, you may want to watch [this talk](https://www.youtube.com/watch?v=Z15cBAuY7Sc) by Soumith Chintala, Facebook AI researcher and core PyTorch contributor.

If you want to learn more PyTorch, you can try this [tutorial](http://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html) or this [learning by examples](http://pytorch.org/tutorials/beginner/pytorch_with_examples.html).

In [106]:
#import 
import torch
import torch.cuda as tc
from torch.autograd import Variable

In [108]:
#copying the tf_ifd to cuda

t_vectors = torch.Tensor(R.astype(np.float32)).cuda()

# Torch implementation

In [127]:
def penalty(W,tol=1e-15):
    """
    Compute the error according the frobenius norm
    """
    return (W<tol).type(tc.FloatTensor)*torch.clamp(W - tol, max=0.)
def gradients(W,H,lam=1):
    """
    Compute the gradient of the loss function 
    return dW, dH
    """
    
    #compute the difference
    err= W.mm(H)  - t_vectors
    
    return err.mm(H.t())+ lam*penalty(W), W.t().mm(err) + lam*penalty(H)


def update(W,H,lr=lr_rate):
    """
    Function to performe one iteration of SGD
    """
    #compute the gradients
    dW, dH = gradients(W,H)
    W.sub_(lr*dW)
    H.sub_(lr*dH)
    
def report(W,H):
    """
    Function to plot the statistics about the matrix
    """
    loss = (t_vectors- W.mm(H)).norm(2)
    print("    loss: {:.4f}, min values = {:.2f},{:.2f}, negative values = {:d}, {:d} ".format(loss, W.min(),H.min(), (W<0).sum(),(H<0).sum()))

In [128]:
#initialistaion in the gpu
t_W = tc.FloatTensor(m,d)
t_H = tc.FloatTensor(d,n)
t_W.normal_(std=0.01).abs_(); 
t_H.normal_(std=0.01).abs_();

In [130]:
lr = 0.05
for i in range(1000): 
    update(t_W,t_H,lr)
    if i % 100 == 0: 
        report(t_W,t_H)
        lr *= 0.9

    loss: 44.3653, min values = -0.01,-0.00, negative values = 1408, 3348 
    loss: 43.7283, min values = -0.10,-0.16, negative values = 3657, 33919 
    loss: 43.7037, min values = -0.13,-0.10, negative values = 3532, 36118 
    loss: 43.7003, min values = -0.09,-0.09, negative values = 3353, 34852 
    loss: 43.7000, min values = -0.06,-0.08, negative values = 3608, 34870 
    loss: 43.7000, min values = -0.05,-0.07, negative values = 3684, 36958 
    loss: 43.7000, min values = -0.05,-0.07, negative values = 3692, 39172 
    loss: 43.7000, min values = -0.05,-0.07, negative values = 3688, 40815 
    loss: 43.7000, min values = -0.05,-0.07, negative values = 3698, 42310 
    loss: 43.7001, min values = -0.05,-0.07, negative values = 3702, 43702 


Let's show the topic

In [131]:
show_topics(t_H.cpu().numpy())  # we realized several transers (gpu---> cpu -- numpy)

['space nasa shuttle launch orbit moon lunar earth',
 'think don people just objective like morality moral',
 'ico bobbe tek bronx beauchaine manhattan sank queens',
 'thanks graphics files image file know program windows',
 'god jesus bible believe christian does atheism belief']